# Image Processing

Import the relevant libraries.

In [21]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, combiner
from astropy import units as u
import warnings
warnings.filterwarnings('ignore')

Load in the median combined bias, dark, and flats for each band.

In [7]:
bias_median = CCDData.read('bias_median.fits', unit = 'adu')
dark_median = CCDData.read('dark_median.fits', unit = 'adu')
Rflat_median = CCDData.read('flat_R_median.fits', unit = 'adu')
Vflat_median = CCDData.read('flat_V_median.fits', unit = 'adu')
Bflat_median = CCDData.read('flat_B_median.fits', unit = 'adu')

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu2 in the FITS file. [astropy.nddata.ccddata]


Now we process the science images using the bias, dark and the relevant flat. The following function will do this for us, and save the image with the 'proc_' prefix at the start of the file name.

In [15]:
def process_scims(sci_images, dark, bias, flat, directory):
    # read the science images and save them to a list
    scim = [CCDData.read(directory + '/' + fn, unit = "adu") for fn in sci_images.files_filtered(PICTTYPE = 1)]
    
    print('Processing', len(scim), 'science images')

    # print the counts (in adu) of the first science image
    print(scim[0])

    # subtract bias from each image
    for idx, thisimage in enumerate(scim): 
        scim[idx] = ccdproc.subtract_bias(thisimage, bias)
        
    print('Counts of first column of first science image after bias subtraction')
    print(scim[0][0,:])

    # subtract dark from each image
    for idx, thisimage in enumerate(scim): 
        scim[idx] = ccdproc.subtract_dark(thisimage, dark, exposure_time = 'EXPTIME', 
                                          exposure_unit = u.second, scale = True)
        
    print('Counts of first column of first science image after dark subtraction')
    print(scim[0][0,:])

    # divide by flat for each image
    for idx, thisimage in enumerate(scim): 
        scim[idx] = ccdproc.flat_correct(thisimage, flat)
        
    print('Counts of first column of first science image after division by flat')
    print(scim[0][0,:])

    
    # Write these images to output files. 
    # We added the proc prefix so we can tell the processed images apart from the raw images. 
    newname = []
    for fn in sci_images.files_filtered(PICTTYPE = 1):
        newname.extend(["proc_" + fn])

    # This image writes the science images out (while including some extra lines to reduce the file size)
    for idx, thisimage in enumerate(scim):
        tempimages = [thisimage]
        temp = ccdproc.Combiner(tempimages,dtype=np.float32).median_combine() 
        temp.meta = thisimage.meta
        temp.write(newname[idx])      # write image with new name to file


In [16]:
directory = 'C11_2022_03_28/'

In [17]:
# load images with '_R_' somewhere in the filename so that we only load in the R-band images
R_images = ccdproc.ImageFileCollection("./" + directory,glob_include = '*_R_*')
process_scims(R_images, dark_median, bias_median, Rflat_median, directory)

Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'.


Processing 30 science images
[[2380 2341 2328 ... 2187 2132 2234]
 [2334 2384 2278 ... 2207 2203 2205]
 [2354 2373 2370 ... 2196 2205 2201]
 ...
 [2256 2307 2277 ... 2125 2228 2181]
 [2276 2191 2311 ... 2184 2146 2202]
 [2274 2241 2239 ... 2172 2167 2187]] adu
Counts of first column of first science image after bias subtraction
[230.  198.  188.5 ...  63.   11.  107. ] adu
Counts of first column of first science image after dark subtraction
[229.03263885 197.06041664 187.62430555 ...  62.61666664  10.71041664
 106.70277777] adu
Counts of first column of first science image after division by flat
[228.7578657  198.06883458 188.62871212 ...  94.56855616  16.17347891
 161.34194771] adu


In [19]:
# load images with '_V_' somewhere in the filename so that we only load in the V-band images
V_images = ccdproc.ImageFileCollection("./" + directory,glob_include = '*_V_*')
process_scims(V_images, dark_median, bias_median, Vflat_median, directory)

Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'.


Processing 30 science images
[[2384 2410 2406 ... 2294 2158 2289]
 [2352 2396 2410 ... 2278 2273 2247]
 [2398 2358 2337 ... 2232 2259 2212]
 ...
 [2327 2171 2250 ... 2261 2260 2238]
 [2256 2279 2334 ... 2246 2238 2332]
 [2316 2285 2323 ... 2196 2290 2215]] adu
Counts of first column of first science image after bias subtraction
[234.  267.  266.5 ... 170.   37.  162. ] adu
Counts of first column of first science image after dark subtraction
[233.03263885 266.06041664 265.62430555 ... 169.61666664  36.71041664
 161.70277777] adu
Counts of first column of first science image after division by flat
[230.4115812  264.56576625 263.50644901 ... 185.24799774  40.09691441
 177.24414328] adu


In [20]:
# load images with '_B_' somewhere in the filename so that we only load in the B-band images
B_images = ccdproc.ImageFileCollection("./" + directory,glob_include = '*_B_*')
process_scims(B_images, dark_median, bias_median, Bflat_median, directory)

Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to  2896022.315 from OBSGEO-[LBH].
Set OBSGEO-Z to -3889419.901 from OBSGEO-[LBH]'.


Processing 40 science images
[[2336 2278 2314 ... 2173 2187 2164]
 [2300 2274 2279 ... 2135 2268 2161]
 [2326 2338 2360 ... 2218 2163 2169]
 ...
 [2208 2210 2178 ... 2169 2174 2270]
 [2211 2206 2190 ... 2165 2278 2096]
 [2222 2206 2211 ... 2135 2186 2219]] adu
Counts of first column of first science image after bias subtraction
[186.  135.  174.5 ...  49.   66.   37. ] adu
Counts of first column of first science image after dark subtraction
[185.03263885 134.06041664 173.62430555 ...  48.61666664  65.71041664
  36.70277777] adu
Counts of first column of first science image after division by flat
[188.3735227  136.00562796 175.79946718 ...  57.87903529  78.68267665
  44.24397481] adu


All of the statistics look good. The counts reduce by about 2100 when we subtract the bias, which makes sense since the median of the bias was around 2100. The counts reduce by about 1 when we subtract the dark, which makes sense given the scaling between the exposure time of the dark and the science images. This is independent of the band, which is as expected since the bias and dark are independent of the filter that we use.

We also examined some images in ds9 in each of the bands and found that dust donuts that were visible in the unprocessed science images were removed in the processed science image, meaning that the flat subtraction has worked.